In [1]:
import nltk
import pandas as pd
import numpy as np
nltk.download("popular")
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import PCA
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
df = pd.read_csv('./final1440_aapl.csv')

In [ ]:
df

,Unnamed: 0,text,sentiment,label,stock_time
0,0,2018 nape global busi confer held februari 7 2...,0.024667,1.0,2017-12-08 05:00:00
1,1,725 s8 25 iphon 8 perhap ngalaxi s8 best smart...,0.091103,-1.0,2017-12-11 05:00:00
2,2,web publish would wise rememb arent kind bad a...,0.016494,1.0,2017-12-12 05:00:00
3,3,fact cobalt mine mainli byproduct metal copper...,0.013467,-1.0,2017-12-13 05:00:00
4,4,snap nyse snap cramer fan cramer trust doesnt ...,0.031292,1.0,2017-12-15 05:00:00
...,...,...,...,...,...
281,281,2019 tast year got scorcher blaze hot melbourn...,0.012952,1.0,2019-01-28 05:00:00
282,282,aapl topic sever report compani offer iphon li...,0.011818,1.0,2019-01-29 05:00:00
283,283,httpswwwyoutubecomwatchv6enwkf1sqog tag aapl a...,0.009776,1.0,2019-01-30 05:00:00
284,284,compani offer iphon line smartphon ipad line m...,0.009638,1.0,2019-01-31 05:00:00


In [ ]:
# Load the text, sentiment, and labels from the dataframe
input_text = df['text']
input_sentiment = df['sentiment']
input_labels = df['label']

# Generate ngrams (2,2) or (2,3) with Count Vectorizer
count_vectorizer = CountVectorizer(max_df=0.8, min_df=0.02, stop_words=stop_words, ngram_range=(2,2))
vectorized_text = count_vectorizer.fit_transform(input_text)

# Use TfIdf to determine the frequency
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
X_tfidf = tfidf_transformer.fit_transform(vectorized_text).toarray()

# Use PCA to reduce dimensionality of the text dataset
pca = PCA(n_components=3)
pca_components = pca.fit_transform(X_tfidf)

# Combine text features with sentiment
final_input = np.hstack((pca_components, np.atleast_2d(input_sentiment).T))

# Create train and test split
X_train, X_test, y_train, y_test = train_test_split(final_input, input_labels, test_size=0.3, random_state=23)

# Scale the data
scaler = preprocessing.MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.fit_transform(X_test)


In [ ]:
# Evaluation for ngrams and sentiment

# Create and train XGBoost model
xgboost_classifier = XGBClassifier()
xgboost_classifier.fit(scaled_X_train, y_train)

# Generate predictions
y_train_predicted = xgboost_classifier.predict(scaled_X_train)
y_test_predicted = xgboost_classifier.predict(scaled_X_test)

# Display train and test accuracy
train_accuracy = accuracy_score(y_train, y_train_predicted)
print('Training accuracy: {0}'.format(train_accuracy))
test_accuracy = accuracy_score(y_test, y_test_predicted)
print('Testing accuracy: {0}'.format(test_accuracy))

# Display F1 score
f1_score_value = f1_score(y_test, y_test_predicted)
print('F1 Score: {0}'.format(f1_score_value))

# Display classification report
classification_report_result = classification_report(y_test, y_test_predicted)
print('Classification report:\n{0}'.format(classification_report_result))


Training accuracy is 0.92
Testing accuracy is 0.5697674418604651
F1 Score is 0.6890756302521007
Classifcation report:               precision    recall  f1-score   support

        -1.0       0.32      0.29      0.30        28
         1.0       0.67      0.71      0.69        58

    accuracy                           0.57        86
   macro avg       0.50      0.50      0.50        86
weighted avg       0.56      0.57      0.56        86



In [ ]:
# Train with ngrams only

# Extract text and labels from the dataframe
input_text = df['text']
input_labels = df['label']

# Generate ngrams (2,2) or (2,3) with Count Vectorizer
count_vectorizer = CountVectorizer(max_df=0.8, min_df=0.02, stop_words=stop_words, ngram_range=(2,2))
X_count = count_vectorizer.fit_transform(input_text)

# Use TfIdf to determine term frequency
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
X_tfidf = tfidf_transformer.fit_transform(X_count).toarray()

# Use PCA to reduce dimensionality of the text dataset
pca = PCA(n_components=3)
final_X_train = pca.fit_transform(X_tfidf)

# Create train and test split
X_train, X_test, y_train, y_test = train_test_split(final_X_train, input_labels, test_size=0.3, random_state=23)

# Scale the data
scaler = preprocessing.MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.fit_transform(X_test)

In [ ]:
# Evaluation for ngrams

# Create and train XGBoost model
xgboost_classifier = XGBClassifier()
xgboost_classifier.fit(scaled_X_train, y_train)

# Generate predictions
y_train_predicted = xgboost_classifier.predict(scaled_X_train)
y_test_predicted = xgboost_classifier.predict(scaled_X_test)

# Display train and test accuracy
train_accuracy = accuracy_score(y_train, y_train_predicted)
print('Training accuracy: {0}'.format(train_accuracy))
test_accuracy = accuracy_score(y_test, y_test_predicted)
print('Testing accuracy: {0}'.format(test_accuracy))

# Display F1 score
f1_score_value = f1_score(y_test, y_test_predicted)
print('F1 Score: {0}'.format(f1_score_value))

# Display classification report
classification_report_result = classification_report(y_test, y_test_predicted)
print('Classification report:\n{0}'.format(classification_report_result))
